# Early time series classification with sktime

Early time series classification (eTSC) is the problem of classifying a time series after as few measurements as possible with the highest possible accuracy. The most critical issue of any eTSC method is to decide when enough data of a time series has been seen to take a decision: Waiting for more data points usually makes the classification problem easier but delays the time in which a classification is made; in contrast, earlier classification has to cope with less input data, often leading to inferior accuracy.

This notebook gives a quick guide to get you started with running eTSC algorithms in sktime.


#### References:

\[1\] Schäfer, P., & Leser, U. (2020). TEASER: early and accurate time series classification. Data mining and knowledge discovery, 34(5), 1336-1362

## Data sets and problem types
The UCR/UEA [time series classification archive](https://timeseriesclassification.com/) contains a large number of example TSC problems that have been used thousands of times in the literature to assess TSC algorithms. Read the data loading documentation and notebooks for details on the sktime data formats and loading data for sktime.

In [1]:
# Imports used in this notebook
import numpy as np

from sktime.classification.early_classification._teaser import TEASER
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import load_arrow_head

In [2]:
# Load default train/test splits from sktime/datasets/data
arrow_train_X, arrow_train_y = load_arrow_head(split="train", return_type="numpy3d")
arrow_test_X, arrow_test_y = load_arrow_head(split="test", return_type="numpy3d")

arrow_test_X.shape

(175, 1, 251)

## Building the TEASER classifier

TEASER \[1\] is a two-tier model using a base classifier to make predictions and a decision making estimator to decide whether these predictions are safe. As a first tier, TEASER requires a TSC algorithm, such as WEASEL, which produces class probabilities as output. As a second tier an anomaly detector is required, such as a one-class SVM.

In [3]:
teaser = TEASER(
    random_state=0,
    classification_points=[25, 50, 75, 100, 125, 150, 175, 200, 251],
    estimator=TimeSeriesForestClassifier(n_estimators=10, random_state=0),
)
teaser.fit(arrow_train_X, arrow_train_y)

TEASER(classification_points=[25, 50, 75, 100, 125, 150, 175, 200, 251],
       estimator=TimeSeriesForestClassifier(n_estimators=10, random_state=0),
       random_state=0)

## Determine the accuracy and earliness on the test data

Commonly accuracy is used to determine the correctness of the predictions, while earliness is used to determine how much of the series is required on average to obtain said accuracy. I.e. for the below values, using just 43% of the full test data, we were able to get an accuracy of 69%.

In [4]:
hm, acc, earl = teaser.score(arrow_test_X, arrow_test_y)

In [5]:
print("Earliness on Test Data %2.2f" % earl)
print("Accuracy on Test Data %2.2f" % acc)
print("Harmonic Mean on Test Data %2.2f" % hm)

Earliness on Test Data 0.43
Accuracy on Test Data 0.69
Harmonic Mean on Test Data 0.62


### Determine the accuracy and earliness on the train data

In [6]:
print("Earliness on Train Data %2.2f" % teaser._train_earliness)
print("Accuracy on Train Data %2.2f" % teaser._train_accuracy)

Earliness on Train Data 0.36
Accuracy on Train Data 0.78


### Comparison to Classification on full Test Data

With the full test data, we would obtain 68% accuracy with the same classifier.

In [7]:
accuracy = (
    TimeSeriesForestClassifier(n_estimators=10, random_state=0)
    .fit(arrow_train_X, arrow_train_y)
    .score(arrow_test_X, arrow_test_y)
)
print("Accuracy on the full Test Data %2.2f" % accuracy)

Accuracy on the full Test Data 0.68


## Classifying with incomplete time series

The main draw of eTSC is the capabilility to make classifications with incomplete time series. sktime eTSC algorithms accept inputs with less time points than the full series length, and output three items: The prediction made, whether the algorithm thinks the prediction is safe and information about the decision such as the time stamp it was made at (sometimes used as an input to provide the algorithm with information from previous decisions).

In [8]:
X = arrow_test_X[:, :, :25]
probas, decisions, states = teaser.predict_proba(X, state_info=None)

print(probas[0])
print(decisions[0])
print(states[0])

[-1 -1 -1]
False
[ 0  0  1 25]


In [9]:
test_points = [50, 75, 100, 125, 150, 175, 200, 251]
final_probas = np.zeros((arrow_test_X.shape[0], np.unique(arrow_test_y).shape[0]))
final_decisions = np.zeros(arrow_test_X.shape[0])

for i in test_points:
    X = arrow_test_X[:, :, :i]
    probas, decisions, states = teaser.predict_proba(X, state_info=states)

    for n in range(arrow_test_X.shape[0]):
        if decisions[n] and final_decisions[n] == 0:
            final_probas[n] = probas[n]
            final_decisions[n] = i